# 读取文件

### 有四种获取数据到TensorFlow程序的方法：

1. **`tf.dataAPI`：轻松构建复杂的输入管道（优选方法，在新版本中）**

2. **`QueueRunner`：基于队列的输入管道从TensorFlow图形开头的文件中读取数据**

3. `Feeding`：运行每一步时，Python代码提供数据

4. 预加载数据：TensorFlow图中的常量或变量包含所有数据（对于小数据集）

### 文件读取流程

![imaged13d0.png](https://miao.su/images/2019/10/15/imaged13d0.png)

* 第一阶段将生成文件名来读取他们并将他们排入文件名队列

* 第二阶段对于文件名的队列，进行出队列实例，并且实行内容的解码

* 第三阶段重新入新的队列，这将是新的样本队列

1. 构造好一个路径 + 文件名的队列

2. 进行文件名队列的读取操作

3. 进行解码操作

4. 放到样本队列当中，进行批处理

### 构造文件队列

将需要读取的文件装入到一个固定的队列当中。

`tf.train.string_input_producer(string_tensor, shuffle=True)`
    
    * string_tensor：含有文件名 + 路径的1阶张量
    
    * num_epochs：过几遍数据，默认无限过数据
    
    * return：文件队列

### 读取文件内容

TensorFlow默认每次只读取一个样本，具体到文本文件读取一行、二进制文件读取指定字节数（最好一个样本）、图片文件读取一张图片、TFRecords默认读取一个example。

* `tf.TextLineReader`：

    * 阅读文本文件逗号分隔值（CSV）格式，默认按行读取
    
    * `return`：读取器实例
    
* `tf.WholeFileReader`：用于读取图片文件

* `tf.TFRecordReader`：读取TFRecordReader文件

* `tf.FixedLengthRecordReader`：二进制文件

    * 要读取每个记录是固定数量字节的二进制文件
    
    * `record_bytes`：整型，指定每次读取（一个样本）的字节数
    
    * `reutrn`：读取器实例
  

> 1. 他们共同的读取的方法：read(file_queue)：从队列中指定数量内容返回一个Tensors元组（key文件名字， value默认的内容（一个样本））
> 2. 由于默认只会读取一个样本，所以通常想要进行批处理。使用`tf.train.batch`或`tf.train.shuffle_batch`进行多样本获取，便于训练时候指定每批次多个样本训练

### 内容解码

对于读取不同的文件类型，内容需要解码操作，解码成统一的文件格式。

* `tf.decode_csv`：解码文本文件内容

* `tf.decode_raw`：解码二进制文件内容，与`tf.FixLengthRecordReader`搭配使用，二进制读取为uint8格式

* `tf.image.decode_jepg(contents)`：将JEPG编码的图像解码为uint8张量；返回uint8张量，3-D形状[height, width, channels]

* `tf.image.decode_png(contents)`：将PNG编码的图像解码为uint8张量；返回张量类型，3-D形状[height, width, channels]

> 解码阶段，默认所有解码都返回uint8类型，如果需要别的类型再进行其他处理

### 批处理

在解码之后，我们可以直接获取默认的一个样本内容了，但是如果想要获取多个样本，这个时候需要结合管道的末尾进行批处理

`tf.train.batch(tensors, batch_size, num_threads=1, capacity=32, name=None)`
    
    * 读取指定大小（个数）的张量
    
    * tensor：可以是包含张量的列表，批处理的内容放在列表中
    
    * batch_size：从列表读取的批处理的大小
    
    * num_threads：进入队列的线程数
    
    * capacity：整数，队列中元素的最大数量
    
    * return：tensors
    
`tf.train.shuffle_batch`